In [1]:
import math
import statistics
import numpy as np
import scipy.stats
import pandas as pd
import matplotlib.pyplot as plt 
from time import perf_counter 

In [2]:
name_cols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
raw_df = pd.read_csv("housing.data", names = name_cols, header=None, delim_whitespace = True)
raw_df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [3]:
print(raw_df.isnull().sum() ) # check for null values
row_num = raw_df.shape[0]
col_num = raw_df.shape[1]
print(f'Number of rows/datapoints: {row_num}')
print(f'Number of columns/features: {col_num}')

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64
Number of rows/datapoints: 506
Number of columns/features: 14


In [4]:
df = pd.DataFrame(raw_df).to_numpy() # convert from dataframe format to numpy format
print(f'last row last column: {df[505][-1]}, type: {type(df[1][-1])}')

last row last column: 11.9, type: <class 'numpy.float64'>


In [5]:
x_features_only = df[:, :-1]  # all features
y_target = df[:, -1]  # only y label
print('x_input shape', x_features_only.shape)
print(y_target.shape)
print(x_features_only[-1])
print(y_target[-10:-1])

x_input shape (506, 13)
(506,)
[4.741e-02 0.000e+00 1.193e+01 0.000e+00 5.730e-01 6.030e+00 8.080e+01
 2.505e+00 1.000e+00 2.730e+02 2.100e+01 3.969e+02 7.880e+00]
[19.7 18.3 21.2 17.5 16.8 22.4 20.6 23.9 22. ]


In [6]:
def dataNorm(X):

    xMerged = X[:,-1].T #output col. switch it's column to a row for vstack later   
    f_transpose = X.T #feature cols. switch the columns to rows for iteration later
    
    for i in f_transpose:  
        arr_transpose = (i - np.min(i)) / np.ptp(i)
        xMerged = np.vstack((xMerged, arr_transpose))
#     print(xMerged)    
#     y_output = xMerged[0] # a row of 'Rings' data points
#     final_merged = np.vstack((xMerged[1:], y_output)) # vstack the 8 features and the output at the bottom of the stack 
    final_merged = xMerged[1:]
    return final_merged.T 
    
x_features_only = dataNorm(x_features_only)
print(x_features_only.shape)
print(x_features_only)

(506, 13)
[[0.00000000e+00 1.80000000e-01 6.78152493e-02 ... 2.87234043e-01
  1.00000000e+00 8.96799117e-02]
 [2.35922539e-04 0.00000000e+00 2.42302053e-01 ... 5.53191489e-01
  1.00000000e+00 2.04470199e-01]
 [2.35697744e-04 0.00000000e+00 2.42302053e-01 ... 5.53191489e-01
  9.89737254e-01 6.34657837e-02]
 ...
 [6.11892474e-04 0.00000000e+00 4.20454545e-01 ... 8.93617021e-01
  1.00000000e+00 1.07891832e-01]
 [1.16072990e-03 0.00000000e+00 4.20454545e-01 ... 8.93617021e-01
  9.91300620e-01 1.31070640e-01]
 [4.61841693e-04 0.00000000e+00 4.20454545e-01 ... 8.93617021e-01
  1.00000000e+00 1.69701987e-01]]


In [7]:
# test unit for joining the x_input and y_target back together

print(y_target.shape)
y_response = y_target.reshape((506,1)) # gotta reshape for concatenate purpose
print(y_response.shape)
x_features_y_response = np.concatenate((x_features_only, y_response), axis=1) # normalised features + y_label
print(x_features_y_response[-1])
print(x_features_y_response.shape)

(506,)
(506, 1)
[4.61841693e-04 0.00000000e+00 4.20454545e-01 0.00000000e+00
 3.86831276e-01 4.73079134e-01 8.02265705e-01 1.25071611e-01
 0.00000000e+00 1.64122137e-01 8.93617021e-01 1.00000000e+00
 1.69701987e-01 1.19000000e+01]
(506, 14)


In [8]:
# concat an all-ones col to x_input features (for y-intercept computation)
# x_in is for gradient descent computation

x_features_ones = np.concatenate([x_features_only, np.ones([np.shape(x_features_only)[0], 1])], axis=1)
print(x_features_ones)
print(x_features_ones.shape)

[[0.00000000e+00 1.80000000e-01 6.78152493e-02 ... 1.00000000e+00
  8.96799117e-02 1.00000000e+00]
 [2.35922539e-04 0.00000000e+00 2.42302053e-01 ... 1.00000000e+00
  2.04470199e-01 1.00000000e+00]
 [2.35697744e-04 0.00000000e+00 2.42302053e-01 ... 9.89737254e-01
  6.34657837e-02 1.00000000e+00]
 ...
 [6.11892474e-04 0.00000000e+00 4.20454545e-01 ... 1.00000000e+00
  1.07891832e-01 1.00000000e+00]
 [1.16072990e-03 0.00000000e+00 4.20454545e-01 ... 9.91300620e-01
  1.31070640e-01 1.00000000e+00]
 [4.61841693e-04 0.00000000e+00 4.20454545e-01 ... 1.00000000e+00
  1.69701987e-01 1.00000000e+00]]
(506, 14)


In [9]:
# concat an all-ones col to x_input features (for y-intercept computation)
# x_in is for gradient descent computation
import pprint as pp
x_features_ones_ylabel = np.concatenate([x_features_only, np.ones([np.shape(x_features_only)[0], 1]), y_response], axis=1)
pp.pprint(x_features_ones_ylabel)
print(x_features_ones_ylabel.shape)

array([[0.00000000e+00, 1.80000000e-01, 6.78152493e-02, ...,
        8.96799117e-02, 1.00000000e+00, 2.40000000e+01],
       [2.35922539e-04, 0.00000000e+00, 2.42302053e-01, ...,
        2.04470199e-01, 1.00000000e+00, 2.16000000e+01],
       [2.35697744e-04, 0.00000000e+00, 2.42302053e-01, ...,
        6.34657837e-02, 1.00000000e+00, 3.47000000e+01],
       ...,
       [6.11892474e-04, 0.00000000e+00, 4.20454545e-01, ...,
        1.07891832e-01, 1.00000000e+00, 2.39000000e+01],
       [1.16072990e-03, 0.00000000e+00, 4.20454545e-01, ...,
        1.31070640e-01, 1.00000000e+00, 2.20000000e+01],
       [4.61841693e-04, 0.00000000e+00, 4.20454545e-01, ...,
        1.69701987e-01, 1.00000000e+00, 1.19000000e+01]])
(506, 15)


In [10]:
#unit test
y_out = x_features_ones_ylabel[:, -1]
y_out.shape

(506,)

In [11]:
# In real life we don't want to code it directly
np.linalg.lstsq(x_features_ones, y_target, rcond=None)[0] # the last in the output is the y-intercept

array([ -9.60975755,   4.64204584,   0.56083933,   2.68673382,
        -8.63457306,  19.88368651,   0.06721501, -16.22666104,
         7.03913802,  -6.46332721,  -8.95582398,   3.69282735,
       -19.01724361,  26.62026758])

In [12]:
# Part 5
def gradient_func(weights, X, y_target):  # Vectorized gradient function
    '''
    Given `weights` - a current "Guess" of what our weights should be
          `X` - matrix of shape (N,D) of input features
          `y_target` - target y values
    Return gradient of each weight evaluated at the current value
    '''
    N, D = np.shape(X)
    y_pred = np.dot(X, weights)  # alternative, use np.matmul()
    error = np.subtract(y_pred, y_target)
    return y_pred, error  # return the gradient of the cost function

In [13]:
def predict(x_test, y_test, w):
# def rmse_func(X, y_target, alpha):
    '''
    Given `X` - matrix of shape (N,D) of input features
          `t` - target y values
    Solves for linear regression weights.
    Return weights after `niter` iterations.
    '''
#     N, D = np.shape(X)                                  # feature matrix has N rows and D cols
#     w = np.zeros([D])                                   # initialize all the weights to zeros based on N cols of feature matrix
    y_pred, error = gradient_func(w, x_test, y_test)   # call the gradient function. get y_pred, error output
    print('y_pred shape:', y_pred.shape)
#     print('error:', error)
    
    rmse = np.sqrt(np.square(np.subtract(y_test,y_pred)).mean()) 
    print('rmse:', rmse)
    return y_pred, rmse  # return the gradient of the cost function

#     for k in range(100000):   # loop over niter counts
       
#         y_pred, error = gradient_func(w, X, y_target)   # call the gradient function. get y_pred, error output
#         dw = np.dot(np.transpose(X), error) / float(N)
#         # -------------------------------------------------------------------------------
#         prev = w                           # assign the previous weight to prev variable
#         w = w - alpha * dw                 # update the weight with the learning rate and gradient change 
#         new = w                            # update the new weight to new variable
#         # ------------------------------------------------------------------------------ 
#         MSE = np.square(np.subtract(y_target,y_pred)).mean() 
#     return np.sqrt(MSE), y_pred

In [18]:
# Part 5
def gradient_descent(X, y_target, alpha, print_every=5000, niter=100000):  # gotta varies the alpha to get the most accurate w
    '''
    Given `X` - matrix of shape (N,D) of input features
          `t` - target y values
    Solves for linear regression weights.
    Return weights after `niter` iterations.
    '''
    N, D = np.shape(X)                                  # feature matrix has N rows and D cols
    w = np.zeros([D])                                   # initialize all the weights to zeros based on N cols of feature matrix
    for k in range(niter):   # loop over niter counts
       
        y_pred, error = gradient_func(w, X, y_target)        # call the gradient function. get y_pred, error output
        dw = np.dot(np.transpose(X), error) / float(N)
        # -------------------------------------------------------------------------------
        prev = w                           # assign the previous weight to prev variable
        w = w - alpha * dw                 # update the weight with the learning rate and gradient change 
        new = w                            # update the new weight to new variable
        # ------------------------------------------------------------------------------        
        if k % print_every == 0:           # for every 5000 count
            if np.all(new-prev) == False:  # when there is no improvement over the previous w, then get the latest optimal value
                print(f"Learning rate (alpha) is: {str(alpha)}")
                print(f'Weight after {k} iteration:\n {str(w)}')
                print()
                break                 
            elif k == 95000:
                print()
    return w

In [19]:
# Part 5
for i in np.arange(1.0, 0.0, -0.1):  # Part 5 main calling block
    print('Running in progress ...')
    w = gradient_descent(X = x_features_ones, y_target = y_target, alpha = round(i,3))
print('Running completed.')
# print('The first learning rate that shows up is the optimal learning rate.\n')
print('The optimal weights:\n', w)


Running in progress ...

Running in progress ...

Running in progress ...

Running in progress ...

Running in progress ...

Running in progress ...
Learning rate (alpha) is: 0.5
Weight after 15000 iteration:
 [ -9.60975755   4.64204584   0.56083933   2.68673382  -8.63457306
  19.88368651   0.06721501 -16.22666104   7.03913802  -6.46332721
  -8.95582398   3.69282735 -19.01724361  26.62026758]

Running in progress ...
Learning rate (alpha) is: 0.4
Weight after 20000 iteration:
 [ -9.60975755   4.64204584   0.56083933   2.68673382  -8.63457306
  19.88368651   0.06721501 -16.22666104   7.03913802  -6.46332721
  -8.95582398   3.69282735 -19.01724361  26.62026758]

Running in progress ...
Learning rate (alpha) is: 0.3
Weight after 25000 iteration:
 [ -9.60975755   4.64204584   0.56083933   2.68673382  -8.63457306
  19.88368651   0.06721501 -16.22666104   7.03913802  -6.46332721
  -8.95582398   3.69282735 -19.01724361  26.62026758]

Running in progress ...
Learning rate (alpha) is: 0.2
Weigh

In [ ]:
# Part 6
for i in np.arange(1.0, 0.0, -0.1):  # Part 6 main calling block
#     alpha = round(i, 3)
    print('Running in progress ...')
    rmse, y_pred = rmse_func(X= x_features_ones, y_target = y_target, alpha = round(i,3))
print('Running completed.')
print('RMSE:', rmse)

In [23]:
# Part 6
def splitCV(X_norm, K): # Split a dataset into k folds
    dataset_split = []
    np.random.shuffle(X_norm) # shuffles the rows in the X_norm matrix
    fold_size = int(len(X_norm) / K) # compute the num of rows per fold
    row_num = X_norm.shape[0]

    for i in range(K):
        if i == K-1:
            fold = np.array(X_norm)
            dataset_split.append(X_norm)
        else:
            dataset_split.append(X_norm[:fold_size])
            X_norm = X_norm[fold_size:]       
    return dataset_split

In [24]:
#Part 6
def CV_Main(x_features_ones_ylabel, cv_num): # k = number of neighbors
    cv_list = []
    X_cv = splitCV(x_features_ones_ylabel, cv_num) # split the data set into K folds = number of parts. X_cv is a list of folds
    print('\nCV_computation ongoing ... ')
    for idx, list_array in enumerate(X_cv): # looping the dataset for cross validation 
        duplicate = X_cv.copy()
        test = list_array
        del duplicate[idx]  # delete the test element from duplicate set, remaining become train elements
        train = duplicate   # remaining elements in duplicate become train set
        train = np.vstack((train)) # convert train stack up vertically
        cv_list.append(np.array([test, train])) #append test and train into a list before return
    return cv_list  # cv_list is a list type containing 2 elements - test and train

In [28]:
## for-loop block code to call the knnCV_Main to perform cross validation 
def main(): 

    
    # MAIN CALL BLOCK for CROSS VALIDATION over 5, 10, 15
    for cv in [5,10, 15]:  # Looping over the cv numbers

        cv5_list = []   # got 2 elements. [0] stores y_pred. [1] stores rmse 
        cv10_list = []  # got 2 elements. [0] stores y_pred. [1] stores rmse
        cv15_list = []  # got 2 elements. [0] stores y_pred. [1] stores rmse

        t1_start = perf_counter() # Start the stopwatch / counter 
        cv_list = CV_Main(x_features_ones_ylabel, cv)
        print(f"-------- CV {cv} --------")

        for num in cv_list:

            test = num[0]
            x_test = test[:, :-1]
            y_test = test[:, -1]

            train = num[1]
            x_train = train[:, :-1]
            y_train = train[:, -1]

            w = gradient_descent(x_train, y_train, alpha=0.5)  # get the fitted weights from x, y train sets 
            y_pred, rmse = predict(x_test, y_test, w)          # apply the w onto the x, y test sets to yield y_pred 

            if cv == 5:
                cv5_list.append(y_pred)
                cv5_list.append(rmse) 
            elif cv == 10:
                cv10_list.append(y_pred)
                cv10_list.append(rmse)
            elif cv == 15:
                cv15_list.append(y_pred)
                cv15_list.append(rmse)

            # GRAB the y_pred to OUPUT folder ...
            # GRAB the rmse to DATA folder  ...

        t1_stop = perf_counter() # Stop the stopwatch / counter 
        print(f'\nElapsed time taken {t1_stop-t1_start} seconds\n') 
    print()
    print('---- Run completed ----')    

if __name__ == "__main__": 
    main() 


CV_computation ongoing ... 
-------- CV 5 --------
y_pred shape: (101,)
rmse: 4.952211774052191
y_pred shape: (101,)
rmse: 4.660991931276736
y_pred shape: (101,)
rmse: 6.282746928741
y_pred shape: (101,)
rmse: 4.750666925642314
y_pred shape: (102,)
rmse: 3.7240153273459575

Elapsed time taken 6.258441300000015 seconds


CV_computation ongoing ... 
-------- CV 10 --------
y_pred shape: (50,)
rmse: 5.395895658229513
y_pred shape: (50,)
rmse: 5.683511187073164
y_pred shape: (50,)
rmse: 5.057686082116991
y_pred shape: (50,)
rmse: 6.0713993869579665
y_pred shape: (50,)
rmse: 4.1333042505262245
y_pred shape: (50,)
rmse: 5.539359774793061
y_pred shape: (50,)
rmse: 4.978180487040395
y_pred shape: (50,)
rmse: 3.6018777038374465
y_pred shape: (50,)
rmse: 4.387777879064978
y_pred shape: (56,)
rmse: 3.6725627043071443

Elapsed time taken 12.188356900000002 seconds


CV_computation ongoing ... 
-------- CV 15 --------
y_pred shape: (33,)
rmse: 3.6662690364025763
y_pred shape: (33,)
rmse: 5.3326725

In [ ]:
# def dataStandard(X):

#     xMerged = X[:,-1].T #output col. switch it's column to a row for vstack later   
#     f_transpose = X[:, :-1].T #feature cols. switch the columns to rows for iteration later
    
#     for i in f_transpose:  
#         arr_transpose = (i - np.average(i)) / np.std(i)
#         xMerged = np.vstack((xMerged, arr_transpose))
        
#     y_output = xMerged[0] # a row of 'Rings' data points
#     final_merged = np.vstack((xMerged[1:], y_output)) # vstack the 8 features and the output at the bottom of the stack 
#     return final_merged.T 
    
# X_norm = dataStandard(X)
# print(X_norm.shape)
# print(len(X_norm))
# X_norm[0:2]